In [ ]:
# Non-Economic Losses Analysis - Setup
import os
import json
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

# Plotting style
sns.set_theme(context="notebook", style="whitegrid")
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "legend.frameon": False,
})

# Constants
INPUT_CSV = "/Users/zhangruotian/Documents/apify_scraper/analysis/merged_all_flood_data.csv"
LOSS = [
    "displacement","education_disruption","health_trauma","social_ties_loss",
    "cultural_ritual_disruption","caregiving_burden","water_food_insecurity",
    "infrastructure_access","psychosocial_distress"
]
PRESENT = [f"loss_{k}_present" for k in LOSS]
CONF = [f"loss_{k}_confidence" for k in LOSS]

# Depth and crowd categorical orders
DEPTH_ORDER = ["none","ankle","knee","waist","vehicle_height","indoor_flood","unknown"]
CROWD_ORDER = ["1","2-5","6-20",">20","unknown"]

print("✅ Setup complete.")


In [ ]:
# Load data and unify timestamps
RAW = pd.read_csv(INPUT_CSV)
DF = RAW.copy()

# Normalize platform label
DF['platform'] = DF.get('source', '').astype(str).str.lower()

# Unify timestamp column `ts` using platform-specific fields

def parse_ts(row):
    src = str(row.get('source', '')).lower()
    if src == 'twitter':
        return pd.to_datetime(row.get('created_at'), errors='coerce', utc=True)
    return pd.to_datetime(row.get('uploaded_at_formatted'), errors='coerce', utc=True)

DF['ts'] = DF.apply(parse_ts, axis=1)
DF['date'] = DF['ts'].dt.date
DF['week'] = DF['ts'].dt.to_period('W').dt.start_time

# Basic normalization for key columns if missing or wrong dtype
for col in PRESENT:
    if col in DF.columns:
        DF[col] = DF[col].astype('bool', errors='ignore')
for col in CONF:
    if col in DF.columns:
        DF[col] = pd.to_numeric(DF[col], errors='coerce')

# Ensure urgency_score exists
if 'urgency_score' in DF.columns:
    DF['urgency_score'] = pd.to_numeric(DF['urgency_score'], errors='coerce').fillna(0).astype(int)

print(f"✅ Loaded rows: {len(DF)} | Valid timestamps: {(DF['ts'].notna()).sum()}")


In [ ]:
# Prepare columns: list-like parsing, categorical orders, derived fields

# Parse list-like columns safely
for col in ['damage_signs', 'context_area']:
    if col in DF.columns:
        DF[col] = DF[col].apply(
            lambda x: x if isinstance(x, list)
            else (json.loads(x) if isinstance(x, str) and x.strip().startswith('[') else ([] if pd.isna(x) else [str(x)]))
        )
    else:
        DF[col] = [[] for _ in range(len(DF))]

# Derived counts
DF['damage_signs_count'] = DF['damage_signs'].apply(len)

# Categorical orders for depth and crowd
if 'water_depth_bin' in DF.columns:
    DF['water_depth_bin'] = DF['water_depth_bin'].astype(CategoricalDtype(DEPTH_ORDER, ordered=True))
else:
    DF['water_depth_bin'] = pd.Categorical(['unknown'] * len(DF), categories=DEPTH_ORDER, ordered=True)

if 'crowd_size_bin' in DF.columns:
    DF['crowd_size_bin'] = DF['crowd_size_bin'].astype(CategoricalDtype(CROWD_ORDER, ordered=True))
else:
    DF['crowd_size_bin'] = pd.Categorical(['unknown'] * len(DF), categories=CROWD_ORDER, ordered=True)

# Relief visible bool normalization
if 'relief_visible' in DF.columns:
    DF['relief_visible'] = DF['relief_visible'].astype('bool', errors='ignore')
else:
    DF['relief_visible'] = False

# Region normalization placeholder (keeps as-is)
if 'region' in DF.columns:
    DF['region'] = DF['region'].astype(str)

print("✅ Columns prepared.")


In [ ]:
# Analysis 1: Frequency & Confidence

# Prepare a tidy table of metrics per loss type
rows = []
for k in LOSS:
    p = DF[f"loss_{k}_present"].mean(skipna=True) if f"loss_{k}_present" in DF.columns else np.nan
    w = (DF[f"loss_{k}_present"] * DF[f"loss_{k}_confidence"]).mean(skipna=True) if f"loss_{k}_confidence" in DF.columns and f"loss_{k}_present" in DF.columns else np.nan
    cbar = DF.loc[DF.get(f"loss_{k}_present", False) == True, f"loss_{k}_confidence"].mean(skipna=True) if f"loss_{k}_confidence" in DF.columns and f"loss_{k}_present" in DF.columns else np.nan
    rows.append({"loss": k, "occurrence_rate": p, "weighted_rate": w, "mean_conf_when_present": cbar})

freq_df = pd.DataFrame(rows).sort_values("weighted_rate", ascending=False)
freq_df.reset_index(drop=True, inplace=True)

# Plot p_k vs ptilde_k
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.4
idx = np.arange(len(freq_df))
ax.barh(idx + 0.2, freq_df['occurrence_rate'], height=bar_width, label='Occurrence rate (p_k)')
ax.barh(idx - 0.2, freq_df['weighted_rate'], height=bar_width, label='Confidence-weighted (ptilde_k)')
ax.set_yticks(idx)
ax.set_yticklabels(freq_df['loss'])
ax.invert_yaxis()
ax.set_xlabel('Rate')
ax.set_title('Loss Type Frequency and Confidence-Weighted Frequency')
ax.legend()
plt.show()

freq_df.head(20)


In [ ]:
# Analysis 2: Pairwise Co-occurrence (Jaccard)

# Build Jaccard matrix over present flags
loss_flags = DF[PRESENT].fillna(False).astype(bool)
labels = [c.replace('loss_','').replace('_present','') for c in loss_flags.columns]

jac = np.zeros((len(labels), len(labels)), dtype=float)
for i in range(len(labels)):
    a = loss_flags.iloc[:, i].to_numpy()
    for j in range(len(labels)):
        b = loss_flags.iloc[:, j].to_numpy()
        inter = (a & b).sum()
        union = (a | b).sum()
        jac[i, j] = (inter / union) if union else 0.0

jac_df = pd.DataFrame(jac, index=labels, columns=labels)

# Plot upper-triangle heatmap
mask = np.triu(np.ones_like(jac_df, dtype=bool), k=1)
plt.figure(figsize=(9, 7))
sns.heatmap(jac_df, mask=~mask, cmap='Blues', vmin=0, vmax=1, cbar_kws={'label':'Jaccard'})
plt.title('Pairwise Co-occurrence (Jaccard) of Loss Types')
plt.tight_layout()
plt.show()

# Top co-occurring pairs
pairs = []
for i in range(len(labels)):
    for j in range(i+1, len(labels)):
        pairs.append((labels[i], labels[j], jac[i, j]))

top_pairs = pd.DataFrame(pairs, columns=['loss_i','loss_j','jaccard']).sort_values('jaccard', ascending=False).head(15)
top_pairs


In [ ]:
# Analysis 3: Relationship with Water Depth / Urgency

# Convert depth bin to numeric level for trend computations
DEPTH_TO_NUM = {v: i for i, v in enumerate(DEPTH_ORDER)}
DF['depth_level_num'] = DF['water_depth_bin'].astype(str).map(DEPTH_TO_NUM).fillna(np.nan)

# Dose-response: p_k by depth
trend_rows = []
for k in LOSS:
    col = f"loss_{k}_present"
    if col in DF.columns:
        grp = DF.groupby('water_depth_bin')[col].mean()
        for depth, val in grp.items():
            trend_rows.append({'loss': k, 'water_depth_bin': depth, 'p_k': val})

dose_df = pd.DataFrame(trend_rows)

# Plot as small multiples
g = sns.relplot(
    data=dose_df, x='water_depth_bin', y='p_k', col='loss', col_wrap=3,
    kind='line', marker='o', facet_kws={'sharey': False}
)
for ax in g.axes.flat:
    ax.set_ylim(0, 1)
    ax.set_xlabel('Water depth bin')
    ax.set_ylabel('Occurrence rate')
plt.suptitle('Dose-Response: Loss occurrence vs Water Depth', y=1.02)
plt.show()

# Urgency vs depth and damage
urg_df = DF[['depth_level_num','damage_signs_count','urgency_score']].copy()
urg_corr = urg_df.corr(method='spearman')
print("Spearman correlations among depth, damage, urgency:\n", urg_corr)

# Partial visuals
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.boxplot(data=DF, x='water_depth_bin', y='urgency_score', ax=axes[0])
axes[0].set_title('Urgency by Depth')
axes[0].set_xlabel('Depth bin')
axes[0].set_ylabel('Urgency (0-5)')

sns.scatterplot(data=DF, x='damage_signs_count', y='urgency_score', alpha=0.4, ax=axes[1])
axes[1].set_title('Urgency vs Damage Signs Count')
axes[1].set_xlabel('Damage signs count')
axes[1].set_ylabel('Urgency (0-5)')
plt.tight_layout()
plt.show()


In [ ]:
# Analysis 4: Temporal Change

# Daily series for each loss: p_k(t)
TS = DF.set_index('ts').sort_index()
TS_daily = TS.resample('D')[PRESENT].mean()
TS_weekly = TS.resample('W')[PRESENT].mean()

# 7-day moving average (centered)
TS_daily_ma7 = TS_daily.rolling(window=7, min_periods=1, center=True).mean()

# Plot 9 facets for daily MA
melt = TS_daily_ma7.reset_index().melt(id_vars='ts', var_name='loss_present', value_name='rate')
melt['loss'] = melt['loss_present'].str.replace('loss_','').str.replace('_present','', regex=False)

g = sns.relplot(
    data=melt, x='ts', y='rate', col='loss', col_wrap=3,
    kind='line', height=2.8, facet_kws={'sharey': False}
)
for ax in g.axes.flat:
    ax.set_ylabel('Rate (7D MA)')
    ax.set_xlabel('Date')
plt.suptitle('Temporal Evolution of Loss Occurrence (7D MA)', y=1.02)
plt.show()

# Lag check: cross-correlation (naive) between infra_access and education_disruption weekly
from pandas import Series

def xcorr(a: Series, b: Series, max_lag=8):
    # Center and fill
    a = a.fillna(0) - a.mean()
    b = b.fillna(0) - b.mean()
    lags = list(range(-max_lag, max_lag+1))
    vals = []
    for L in lags:
        if L < 0:
            vals.append(a.shift(-L).corr(b))
        else:
            vals.append(a.corr(b.shift(L)))
    return pd.DataFrame({'lag': lags, 'xcorr': vals})

ia = TS_weekly['loss_infrastructure_access_present']
ed = TS_weekly['loss_education_disruption_present']
xc = xcorr(ia, ed, max_lag=12)

plt.figure(figsize=(7,3))
plt.stem(xc['lag'], xc['xcorr'], use_line_collection=True)
plt.axhline(0, color='k', lw=0.8)
plt.title('Cross-correlation: infra_access vs education_disruption (weekly)')
plt.xlabel('Lag (weeks); positive means infra leads')
plt.ylabel('Correlation')
plt.tight_layout()
plt.show()

xc.sort_values('xcorr', ascending=False).head(3)


In [ ]:
# Additional Analyses: Visual Cues, Relief, Demographics, Sentiment, Recovery

# 1) Crowd size × infrastructure_access
if 'loss_infrastructure_access_present' in DF.columns:
    infra_crowd = (
        DF.groupby('crowd_size_bin')['loss_infrastructure_access_present']
          .mean().reindex(CROWD_ORDER)
    )
    plt.figure(figsize=(6,4))
    sns.barplot(x=infra_crowd.index, y=infra_crowd.values, color='#4C72B0')
    plt.ylim(0,1)
    plt.title('Infrastructure Access Disruption by Crowd Size')
    plt.xlabel('Crowd size bin')
    plt.ylabel('Occurrence rate')
    plt.show()

# 2) Relief visibility by depth and context
relief_by_depth = DF.groupby('water_depth_bin')['relief_visible'].mean()
plt.figure(figsize=(6,4))
sns.barplot(x=relief_by_depth.index.astype(str), y=relief_by_depth.values, color='#55A868')
plt.ylim(0,1)
plt.title('Relief Visible by Depth')
plt.xlabel('Depth bin')
plt.ylabel('Share with relief visible')
plt.show()

# Relief actor type distribution where relief is visible
if 'relief_actor_type' in DF.columns:
    actors = DF.loc[DF['relief_visible'] == True, 'relief_actor_type'].fillna('unknown').value_counts(normalize=True)
    actors.plot(kind='bar', color='#C44E52', figsize=(6,4), title='Relief Actor Types (where relief visible)')
    plt.ylabel('Share')
    plt.ylim(0,1)
    plt.show()

# 3) Damage sign frequencies
from collections import Counter
all_signs = Counter(s for row in DF['damage_signs'] for s in row if s and s != 'none')
damage_freq = (pd.Series(all_signs).sort_values(ascending=False) if all_signs else pd.Series(dtype=float))
damage_freq.head(15)


In [ ]:
# Composite Indicators: NELI and VISI

# NELI: sum_k 1 * present_k * conf_k (uniform weights)
conf_cols = [f'loss_{k}_confidence' for k in LOSS]
pres_cols = [f'loss_{k}_present' for k in LOSS]

# Defensive: fill missing with zeros/False
conf_mat = DF[conf_cols].fillna(0.0)
pres_mat = DF[pres_cols].fillna(False).astype(bool)
NELI = (conf_mat.values * pres_mat.values).sum(axis=1)
DF['NELI'] = NELI

# VISI: depth_level_num + damage_signs_count + crowd_size_num + 1*relief_visible
crowd_to_num = {v: i for i, v in enumerate(CROWD_ORDER)}
DF['crowd_size_num'] = DF['crowd_size_bin'].astype(str).map(crowd_to_num)
DF['VISI'] = DF['depth_level_num'].fillna(0) + DF['damage_signs_count'].fillna(0) + DF['crowd_size_num'].fillna(0) + DF['relief_visible'].astype(int)

# Distributions
fig, axes = plt.subplots(1,2, figsize=(12,4))
sns.histplot(DF['NELI'], bins=30, ax=axes[0])
axes[0].set_title('NELI Distribution')

sns.histplot(DF['VISI'], bins=30, ax=axes[1])
axes[1].set_title('VISI Distribution')
plt.tight_layout()
plt.show()

# Correlation and scatter by scene/context (high level)
cols = ['NELI','VISI','urgency_score','depth_level_num','damage_signs_count']
print(DF[cols].corr(method='spearman'))

sns.scatterplot(data=DF, x='VISI', y='NELI', alpha=0.3)
plt.title('NELI vs VISI')
plt.show()


In [ ]:
# Data Quality & Validation

# Visual Verifiability Score per plan
# +0.2 if water_depth_bin in set, +0.2 if damage_signs includes any of listed, +0.2 if any loss present with conf>=0.8,
# +0.2 if scene_type is ground_outdoor/indoor, +0.2 if context_area != 'unknown'

def compute_verifiability(row):
    score = 0.0
    # depth
    if str(row.get('water_depth_bin','unknown')) in ['none','ankle','knee','waist','vehicle_height','indoor_flood']:
        score += 0.2
    # damage signs
    ds = row.get('damage_signs', []) or []
    allowed = {'road_blocked','house_inundated','bridge_damage','school_closed_sign','clinic_closed_sign','power_outage_sign'}
    if any(s in allowed for s in ds):
        score += 0.2
    # loss present with conf>=0.8
    ok = False
    for k in LOSS:
        if row.get(f'loss_{k}_present') and (row.get(f'loss_{k}_confidence', 0) >= 0.8):
            ok = True; break
    if ok:
        score += 0.2
    # scene type
    scene_ok = False
    for s in ['scene_ground_outdoor','scene_indoor']:
        if s in row and bool(row[s]):
            scene_ok = True
    if scene_ok:
        score += 0.2
    # context area known
    ctx = row.get('context_area', []) or []
    if any(c != 'unknown' for c in ctx):
        score += 0.2
    return score

DF['verifiability'] = DF.apply(compute_verifiability, axis=1)

# Histogram and by platform
fig, axes = plt.subplots(1,2, figsize=(12,4))
sns.histplot(DF['verifiability'], bins=[0,0.2,0.4,0.6,0.8,1.0], ax=axes[0])
axes[0].set_title('Visual Verifiability Distribution')

sns.boxplot(data=DF, x='platform', y='verifiability', ax=axes[1])
axes[1].set_title('Verifiability by Platform')
plt.tight_layout()
plt.show()

# Unknown proportions
unknown_depth = (DF['water_depth_bin'].astype(str) == 'unknown').mean()
unknown_area = DF['context_area'].apply(lambda xs: len(xs)==0 or all(x=='unknown' for x in xs)).mean()
unknown_scene = (DF.get('scene_aerial', False) == False) & (DF.get('scene_ground_outdoor', False) == False) & (DF.get('scene_indoor', False) == False)
unknown_scene = unknown_scene.mean() if hasattr(unknown_scene, 'mean') else np.nan

print({
    'unknown_water_depth_bin': unknown_depth,
    'unknown_context_area': unknown_area,
    'unknown_scene_type_all_false': unknown_scene
})


In [ ]:
# Stratified: Platform comparison (Twitter vs TikTok)

platform_metrics = []
for plat, sub in DF.groupby('platform'):
    metrics = {'platform': plat, 'n': len(sub)}
    for k in LOSS:
        p = sub.get(f'loss_{k}_present', pd.Series(dtype=float)).mean()
        w = (sub.get(f'loss_{k}_present', 0) * sub.get(f'loss_{k}_confidence', 0)).mean()
        metrics[f'{k}_p'] = p
        metrics[f'{k}_w'] = w
    metrics['relief_visible_share'] = sub.get('relief_visible', pd.Series(dtype=float)).mean()
    metrics['mean_urgency'] = sub.get('urgency_score', pd.Series(dtype=float)).mean()
    platform_metrics.append(metrics)

plat_df = pd.DataFrame(platform_metrics)
plat_df

# Visual: selected losses and relief
sel = ['infrastructure_access','water_food_insecurity','education_disruption','psychosocial_distress']
plot_df = (
    plat_df.melt(id_vars=['platform','n'], value_vars=[f'{k}_p' for k in sel], var_name='metric', value_name='rate')
)
plot_df['loss'] = plot_df['metric'].str.replace('_p','', regex=False)

plt.figure(figsize=(8,5))
sns.barplot(data=plot_df, x='loss', y='rate', hue='platform')
plt.ylim(0,1)
plt.title('Platform comparison: key loss occurrence rates')
plt.ylabel('Rate')
plt.show()

plt.figure(figsize=(6,4))
sns.barplot(data=plat_df, x='platform', y='relief_visible_share')
plt.ylim(0,1)
plt.title('Relief visibility by platform')
plt.ylabel('Share')
plt.show()


In [ ]:
# Stratified: Regional comparison

if 'region' in DF.columns:
    # Keep top-N by count
    topN = 12
    region_sizes = DF['region'].value_counts()
    top_regions = set(region_sizes.head(topN).index)
    REG = DF[DF['region'].isin(top_regions)].copy()

    # Heatmap of key loss rates by region
    key_losses = ['infrastructure_access','water_food_insecurity','education_disruption','psychosocial_distress']
    mat = REG.groupby('region')[[f'loss_{k}_present' for k in key_losses]].mean().reindex(region_sizes.head(topN).index)

    plt.figure(figsize=(10,6))
    sns.heatmap(mat, annot=True, fmt='.2f', cmap='YlOrBr', vmin=0, vmax=1)
    plt.title('Regional comparison: key loss occurrence rates (top regions by count)')
    plt.tight_layout()
    plt.show()

    # Faceted time series by region for infrastructure access
    reg_ts = (
        REG.set_index('ts').sort_index()
           .groupby('region')['loss_infrastructure_access_present']
           .resample('W').mean().reset_index()
    )
    g = sns.relplot(
        data=reg_ts, x='ts', y='loss_infrastructure_access_present', col='region', col_wrap=4,
        kind='line', height=2.2, facet_kws={'sharey': False}
    )
    for ax in g.axes.flat:
        ax.set_ylabel('Rate (weekly)')
        ax.set_xlabel('')
    plt.suptitle('Infrastructure Access over Time by Region (Weekly)', y=1.02)
    plt.show()
else:
    print('Region column not found.')


### Findings (Draft)

- Frequency & Confidence: Highlight top-3 by weighted rate; note low-confidence/rare classes.
- Co-occurrence: Note strongest pairs (e.g., infra_access ↔ water_food_insecurity if observed).
- Depth & Urgency: Urgency rises with depth/damage; list Spearman signs.
- Temporal: Report lag with highest cross-corr between infra_access and education_disruption.
- Additional: Relief appears more at deeper levels; actor type mix shown.
- Composite: NELI correlates positively with VISI and urgency.
- Platform: Contrast key loss rates and relief visibility.
- Region: Surface regional heterogeneity; show top varying regions.

Refine after reviewing actual plots/tables above.
